### Information Retrieval   |    Assignment 1-B

Step 1:

In [1]:
import pandas as pd
import numpy as np
import sys
import os
import random as rnd

In [43]:
df_raw = pd.read_table('YandexRelPredChallenge.txt', delimiter="\t",index_col=None, 
                 header=0,names=['SessionID', 'TimePassed','TypeOfAction', 'QueryID','RegionID', 'ListOfURLs', 'URLID',
                                 '1','2','3','4','5',"6",'7','8'])
df_raw

In [2]:
# Open file    
fileHandler = open ('YandexRelPredChallenge.txt', "r")
# Get list of all lines in file
listOfLines = fileHandler.readlines()
# Close file 
fileHandler.close()
#Empty frame to fill in
index=[x for x in range(0,100000)]
df = pd.DataFrame(index=index, columns=['SessionID', 'TimePassed',
                 'TypeOfAction', 'QueryID','RegionID', 'ListOfURLs', 'URLID'])
df.name='data'

# Iterate over the lines and fill in the dataframe
for ind, line in zip(index, listOfLines):
#     print('index')
    line = line.strip().split()
    if line[2]=='Q':

        df.loc[ind] = pd.Series({'SessionID':line[0], 'TimePassed':line[1], 'TypeOfAction':line[2],
                                     'QueryID':line[3],'RegionID':line[4], 'ListOfURLs':line[5:], 'URLID':np.nan})
    else:
        df.loc[ind] = pd.Series({'SessionID':line[0], 'TimePassed':line[1], 'TypeOfAction':line[2],
                                     'QueryID':np.nan,'RegionID':np.nan, 'ListOfURLs':np.nan, 'URLID':line[3:]})

In [3]:
# Getting the types right. 
df.SessionID = df.SessionID.astype(int)
df.TimePassed = df.TimePassed.astype(int)
df.TypeOfAction = df.TypeOfAction.astype(str)
df.RegionID = df.RegionID.astype(float)
df.TimePassed = df.TimePassed.astype(float)
def parse_ulrid(x):
    if type(x)==list:
        return x[0]
    else:
        return x
df.URLID = df.URLID.apply(parse_ulrid)
df.URLID = df.URLID.astype(float)

Create new relevance:

So lets say that our experimental system is what Yandex return as a list now. Production or the system that is going to 'suck' (sort of a dumb system that we are going to compare against) will be constructed as shuffling each ranking for a query. 

Now, other approach, probably the better one: take the initial order as the 'benchmark' create only list of 0,1 (relevnace scores) for each system.
    - rember that we are interested only in three [3] first results.

In [34]:
# Create the relevance score ranking (those tables = that we called 'runs')
def relevance_list(l,acc=0.5, res_length=3):
    """ Control the 'accuracy'/'performance' with acc param. Only lists of length 3. """
    if type(l)==list:
        res = list()
        for x in l:
            if rnd.random()<acc:
                res.append((x, 1))
            else:
                res.append((x, 0))
        return rnd.sample(res, res_length)
    else:
        return l

df['id_rel_e'] = df.ListOfURLs.apply(relevance_list, acc=0.7) 
df['id_rel_p'] = df.ListOfURLs.apply(relevance_list) 

In [35]:
def extract_rel(l):
    if type(l) == list:
        res = list()
        for tup in l:
            res.append(tup[1])
        return res
    else:
        return l
def extract_id(l):
    if type(l) == list:
        res = list()
        for tup in l:
            res.append(tup[0])
        return res
    else:
        return l
df['rel_e'] = df.id_rel_e.apply(extract_rel)
df['rel_p'] = df.id_rel_p.apply(extract_rel)
df['id_e'] = df.id_rel_e.apply(extract_id)
df['id_p'] = df.id_rel_p.apply(extract_id)

Step 2: Calculate ERR

see the formuala for ERR at: http://olivier.chapelle.cc/pub/err.pdf

In [36]:
def ERR(l):
    """ List or relevance scores. """ 
    if type(l)==list:
        theta = [ (2**i - 1)/(2) for i in [0,1]]
        err=0
        for rank in range(len(l)):
            prod = 1
            for idx in range(rank):
                prod*=(1-theta[l[idx]])
            prod*=(1/(rank+1))*theta[l[rank]]
            err+=prod
        return err
    else:
        return l
df['err_e'] = df.rel_e.apply(ERR)
df['err_p'] = df.rel_p.apply(ERR)

Get the deltas

In [40]:
# Test to see if ERR calculation is implemented correctly and if E outperfroms P 
import math 
df['diff_e_p'] = df.err_e - df.err_p
def count_t(x):
    if not math.isnan(x):
        if x>0:
            return 1
        else:
            return 0 
    else:
        return x 
df['temp_count'] = df.diff_e_p.apply(count_t)
df.loc[:,['diff_e_p', 'temp_count']]
df.temp_count.value_counts()
df.drop(columns=['diff_e_p','temp_count'],inplace=True)

### Step 4.1: Estimating the params of click models: Random Click Model

In [179]:
# Get all the doc ids
data = df.drop(columns=['id_rel_e', 'id_rel_p', 'rel_e', 'rel_p', 'id_e',
       'id_p', 'err_e', 'err_p'])
data.QueryID = data.QueryID.astype(float)
list_doc_ids_raw = data.ListOfURLs[df.ListOfURLs.isnull()!=True].values
res = list()
for i in list_doc_ids_raw:
    res += i
all_doc_ids = res.copy()
# Get all the clicks
list_clicks_ids = data.URLID[df.URLID.isnull()!=True].values

# rho = probability of a doc to be clicked.
rho = len(list_clicks_id)/len(all_doc_ids)
print(rho)

0.13445559411047547


### Step 4.2: Estimating the params of click models: Posistion-based Model

Perform EM steps (update assignments ; update params):
    
    1) initialize alpha and gamma at random 
    2) For each time step:
        - Expecation: do not really have to do this step [?], what we want to see at the end: P(C|u) 
            - calculate P(C|u) for each u (u is from 1 to 3 = positions in the shown list)
            - " P(C|u) = P(A_uq|E,u)P(E|u) "
                
        - Maximization: update alpha and gamma params
        


In [220]:
# Create the tables with all the params
n_docs = 3
unique_queries = data.QueryID[data.QueryID.isnull()!=True].astype(int).unique()
n_queries = len(unique_queries) 
alphas = np.random.rand(n_docs)
gammas = np.random.rand(n_docs*n_queries).reshape((n_docs,n_queries))

In [254]:
# number of sessions
sessions = data.SessionID.unique()

In [288]:
data[data.SessionID==8]

,SessionID,TimePassed,TypeOfAction,QueryID,RegionID,ListOfURLs,URLID
76,8,0.0,Q,3.0,0.0,"[19, 44, 5, 53, 41, 56, 58, 40, 3, 31]",NaN
77,8,24.0,Q,126.0,0.0,"[1199, 1195, 1194, 56, 19, 1196, 1200, 1198, 3...",NaN
78,8,51.0,C,NaN,NaN,NaN,1200.0
79,8,175.0,Q,747.0,0.0,"[6749, 6747, 6742, 6748, 6746, 6745, 6741, 674...",NaN
80,8,180.0,C,NaN,NaN,NaN,6749.0
81,8,381.0,Q,1511.0,0.0,"[13411, 13409, 1199, 13415, 6748, 13413, 13412...",NaN
82,8,391.0,C,NaN,NaN,NaN,13415.0
83,8,436.0,C,NaN,NaN,NaN,13413.0
84,8,1153.0,Q,4350.0,0.0,"[37768, 37765, 37772, 37773, 37769, 37764, 377...",NaN
85,8,1162.0,Q,4393.0,0.0,"[38123, 38129, 38127, 38126, 1194, 38131, 3813...",NaN


In [290]:
# check whether in a session a doc(u) has been clicked on not:
for session in sessions:
    data_session = data[data.SessionID==session]
    click_ids =  data_session.URLID[data_session.URLID.isnull()!=True].values
    first_3_docs = data_session.ListOfURLs[data_session.ListOfURLs.isnull()!=True]
    first_3_docs = first_3_docs.apply(lambda x:list(map(int,x[:3]))).values
    first_3_docs = reduce(lambda x,y:x+y, first_3_docs)
    click_list = list()
    print(session)
    print(click_ids)
#     for click in click_ids:
#         if click in first_3_docs:
#             click_list.append(1)
#         else:
#             click_list.append(0)
#     print(click_list)
    break

0
[17562.  1627.  1626.]


In [238]:
# Write the EM algo,
# if alphas are hardcoded, then  :
    # Get sessions with Query(Q) and doc(u) => will do implicit, coz can use pandas slicing
for i in range(0,100):
    for gamma_ind in range(len(gammas)):
        summ = 0
        for session_i in sessions:
            temp_sum = 
        temp_gamma = 1/len(sessions)*summ
#     gammas = 

In [250]:
arr = list(test.groupby(by="SessionID").count().index)
arr

[3]